In [1]:
import random
import pandas as pd
from datetime import datetime

# Temporary workflow
# (1) Download csv with `heroku run make exportdb && make importdb`, load it here
# (2) save links to ~/Downloads/links.txt in onetab format
# (3) run this script (or otherwise modify the CSV as desired)
# (4) Upload back up with `make csv_to_s3`
# (5) Run `heroku run make importdb` to get it to production
links = pd.read_csv('data/export.csv')
links

,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet
0,0,http://moralitylab.bc.edu/wp-content/uploads/2...,Moral Realism As Moral Motivation: The Impact ...,"""People disagree about whether 'moral facts' a...",moralitylab.bc.edu,2016-12-06 05:00:00+00,2016-12-06 05:00:00+00,1.0,ethics,custom,17.0,0.0
1,1,http://www.jefftk.com/news/2013-04-01,The Unintuitive Power Laws of Giving,"""Why give globally? Why give money? Why health...",www.jefftk.com,2016-12-07 05:00:00+00,2016-12-07 05:00:00+00,1.0,giving,custom,59.0,0.0
2,2,http://calnewport.com/blog/2013/03/24/how-to-w...,How to Write Six Important Papers a Year witho...,"""I'm fascinated by people who produce a large ...",calnewport.com,2016-12-08 05:00:00+00,2016-12-08 05:00:00+00,1.0,productivity,custom,56.0,0.0
3,3,http://www.freakonomics.com/2012/10/31/an-alte...,An Alternative to Democracy,"""In Glen's voting mechanism\, every voter can ...",www.freakonomics.com,2016-12-09 05:00:00+00,2016-12-09 05:00:00+00,1.0,politicalscience,custom,29.0,0.0
4,4,http://www.wired.com/vanish/2009/11/ff_vanish2...,Writer Evan Ratliff Tried to Vanish: Here's Wh...,"""August 13\, 6:40 PM: I'm driving East out of ...",www.wired.com,2016-12-10 05:00:00+00,2016-12-10 05:00:00+00,1.0,technology,custom,92.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
78439,134994,https://cset.georgetown.edu/article/visa-restr...,Visa Restrictions on Chinese Students Endanger...,\N,\N,2021-11-03 11:39:44.36062+00,2021-11-03 11:39:44.36062+00,NaN,\N,CSET,95.0,0.0
78440,134995,https://www.slowboring.com/p/we-need-to-try-ha...,We need to try harder to prevent the next pand...,\N,\N,2021-11-03 11:39:44.362176+00,2021-11-03 11:39:44.362176+00,NaN,\N,Yglesias,69.0,0.0
78441,134996,https://globalguessing.com/manas-chawla-ggwp/,Manas Chawla on Forecast Accuracy in the Geopo...,\N,\N,2021-11-03 11:39:44.363679+00,2021-11-03 11:39:44.363679+00,NaN,\N,GlobalGuessing,51.0,0.0
78442,134997,https://www.cold-takes.com/unraveling-the-evid...,Unraveling the evidence about violence among v...,\N,\N,2021-11-03 11:39:44.365231+00,2021-11-03 11:39:44.365231+00,NaN,\N,Holden,75.0,0.0


In [2]:
links['added'].max()

'2021-11-03 11:39:44.366946+00'

In [3]:
links[(links['aggregator'] == 'Custom') & (links['liked'] != '1') & (links['liked'] != '0') & (links['liked'] != '-1')]['id']

/Users/peterhurford/.virtualenvs/dev/lib/python3.8/site-packages/pandas/core/computation/expressions.py:68: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return op(a, b)
/Users/peterhurford/.virtualenvs/dev/lib/python3.8/site-packages/pandas/core/computation/expressions.py:68: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return op(a, b)
/Users/peterhurford/.virtualenvs/dev/lib/python3.8/site-packages/pandas/core/computation/expressions.py:68: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return op(a, b)


1299       1306
1300       1307
1301       1308
1302       1309
1303       1310
          ...  
78115    134668
78116    134669
78117    134670
78118    134671
78129    134683
Name: id, Length: 3786, dtype: int64

In [4]:
link_file = open('/Users/peterhurford/Downloads/links.txt', 'r')
new_links = link_file.readlines()

In [5]:
def new_entry(links, url, title):
    return {'id': links['id'].max() + 1,
            'url': str(url).strip().replace(',', ''),
            'title': str(title).strip().replace(',', ''),
            'summary': '\\N',
            'domain': '\\N',
            'added': str(datetime.now()),
            'modified': str(datetime.now()),
            'liked': '\\N',
            'category': '\\N',
            'aggregator': 'Custom',
            'seed': random.randint(1, 100),
            'tweet': 0}

for i, new_link in enumerate(new_links):
    if i % 100 == 0:
        print('...{}/{}'.format(i + 1, len(new_links)))
    if ' | ' in new_link:
        new_link = new_link.replace('    - ', '').replace('\n', '')
        entry = new_entry(links, new_link.split(' | ')[0], new_link.split(' | ')[1])
        links = links.append(entry, ignore_index=True)

links

...1/418
...101/418
...201/418
...301/418
...401/418


,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet
0,0,http://moralitylab.bc.edu/wp-content/uploads/2...,Moral Realism As Moral Motivation: The Impact ...,"""People disagree about whether 'moral facts' a...",moralitylab.bc.edu,2016-12-06 05:00:00+00,2016-12-06 05:00:00+00,1,ethics,custom,17.0,0.0
1,1,http://www.jefftk.com/news/2013-04-01,The Unintuitive Power Laws of Giving,"""Why give globally? Why give money? Why health...",www.jefftk.com,2016-12-07 05:00:00+00,2016-12-07 05:00:00+00,1,giving,custom,59.0,0.0
2,2,http://calnewport.com/blog/2013/03/24/how-to-w...,How to Write Six Important Papers a Year witho...,"""I'm fascinated by people who produce a large ...",calnewport.com,2016-12-08 05:00:00+00,2016-12-08 05:00:00+00,1,productivity,custom,56.0,0.0
3,3,http://www.freakonomics.com/2012/10/31/an-alte...,An Alternative to Democracy,"""In Glen's voting mechanism\, every voter can ...",www.freakonomics.com,2016-12-09 05:00:00+00,2016-12-09 05:00:00+00,1,politicalscience,custom,29.0,0.0
4,4,http://www.wired.com/vanish/2009/11/ff_vanish2...,Writer Evan Ratliff Tried to Vanish: Here's Wh...,"""August 13\, 6:40 PM: I'm driving East out of ...",www.wired.com,2016-12-10 05:00:00+00,2016-12-10 05:00:00+00,1,technology,custom,92.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
78815,135370,https://gitlab.com/dsbowen/conditional-inferen...,examples/bayes_primer.ipynb · master · dsbowen...,\N,\N,2021-11-03 06:44:06.529328,2021-11-03 06:44:06.529341,\N,\N,Custom,93.0,0.0
78816,135371,https://www.eurogroupforanimals.org/news/membe...,Members States call for ambitious animal welfa...,\N,\N,2021-11-03 06:44:06.539342,2021-11-03 06:44:06.539354,\N,\N,Custom,26.0,0.0
78817,135372,https://chrisblattman.com/2007/12/12/how-to-ge...,How to get a PhD *and* save the world - Chris ...,\N,\N,2021-11-03 06:44:06.551245,2021-11-03 06:44:06.551256,\N,\N,Custom,86.0,0.0
78818,135373,https://thebulletin.org/2021/09/chinas-nuclear...,China’s nuclear missile silo expansion: From m...,\N,\N,2021-11-03 06:44:06.561783,2021-11-03 06:44:06.561796,\N,\N,Custom,47.0,0.0


In [6]:
links[(links['aggregator'] == 'Custom') & (links['liked'] != '1') & (links['liked'] != '0') & (links['liked'] != '-1')]

,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet
1299,1306,http://blog.beeminder.com/trust/,Trusting Your Divided Self | Beeminder Blog,\N,blog.beeminder.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,0,productivity,Custom,10.0,0.0
1300,1307,http://powlyglot.com/fi3m-review-part-1-focus/,"""What I Learned from Fluent in 3 Months\, Part...",\N,powlyglot.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,0,productivity,Custom,9.0,0.0
1301,1308,http://effective-altruism.com/ea/u8/ea_intervi...,"""EA Interview Series\, February 2016: John Sal...",\N,effective-altruism.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,0,effectivealtruism,Custom,85.0,0.0
1302,1309,https://fivethirtyeight.com/features/most-2020...,Most 2020 Candidates Have Something In Common:...,\N,fivethirtyeight.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,1,politicalscience,Custom,82.0,0.0
1303,1310,http://www.smartpassiveincome.com/backlinking-...,THE Backlinking Strategy That Works – 2014 and...,\N,www.smartpassiveincome.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,0,entrepreneurship,Custom,59.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
78815,135370,https://gitlab.com/dsbowen/conditional-inferen...,examples/bayes_primer.ipynb · master · dsbowen...,\N,\N,2021-11-03 06:44:06.529328,2021-11-03 06:44:06.529341,\N,\N,Custom,93.0,0.0
78816,135371,https://www.eurogroupforanimals.org/news/membe...,Members States call for ambitious animal welfa...,\N,\N,2021-11-03 06:44:06.539342,2021-11-03 06:44:06.539354,\N,\N,Custom,26.0,0.0
78817,135372,https://chrisblattman.com/2007/12/12/how-to-ge...,How to get a PhD *and* save the world - Chris ...,\N,\N,2021-11-03 06:44:06.551245,2021-11-03 06:44:06.551256,\N,\N,Custom,86.0,0.0
78818,135373,https://thebulletin.org/2021/09/chinas-nuclear...,China’s nuclear missile silo expansion: From m...,\N,\N,2021-11-03 06:44:06.561783,2021-11-03 06:44:06.561796,\N,\N,Custom,47.0,0.0


In [7]:
def clean(txt):
    if not isinstance(txt, str):
        return txt
    txt = txt.replace('&nbsp;', '')
    txt = txt.replace('&ldquo;', '"')
    txt = txt.replace('&rdquo;', '"')
    txt = txt.replace('&lsquo;', '\'')
    txt = txt.replace('&rsquo;', '\'')
    txt = txt.replace('&mdash;', '-')
    txt = txt.replace('&ndash;', '-')
    for i in range(10):
        txt = txt.replace('"""', '"')
        txt = txt.replace('\'\'\'', '\'')
    txt = txt.replace('n"t', 'n\'t')
    txt = txt.replace('n""t', 'n\'t')
    return txt

links = links[links['id'].notnull()]   # Drop empty column
links['id'] = links['id'].astype(int)  # Fix float ID issue
links['seed'] = links['seed'].astype(int)
links['tweet'] = links['tweet'].apply(lambda x: 0 if str(x) == '\\N' else str(x).split('.')[0]).astype(int)
links['summary'] = links['summary'].apply(clean)
links['title'] = links['title'].apply(clean)
links = links.sort_values('id')
links

,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet
0,0,http://moralitylab.bc.edu/wp-content/uploads/2...,Moral Realism As Moral Motivation: The Impact ...,"""People disagree about whether 'moral facts' a...",moralitylab.bc.edu,2016-12-06 05:00:00+00,2016-12-06 05:00:00+00,1,ethics,custom,17,0
1,1,http://www.jefftk.com/news/2013-04-01,The Unintuitive Power Laws of Giving,"""Why give globally? Why give money? Why health...",www.jefftk.com,2016-12-07 05:00:00+00,2016-12-07 05:00:00+00,1,giving,custom,59,0
2,2,http://calnewport.com/blog/2013/03/24/how-to-w...,How to Write Six Important Papers a Year witho...,"""I'm fascinated by people who produce a large ...",calnewport.com,2016-12-08 05:00:00+00,2016-12-08 05:00:00+00,1,productivity,custom,56,0
3,3,http://www.freakonomics.com/2012/10/31/an-alte...,An Alternative to Democracy,"""In Glen's voting mechanism\, every voter can ...",www.freakonomics.com,2016-12-09 05:00:00+00,2016-12-09 05:00:00+00,1,politicalscience,custom,29,0
4,4,http://www.wired.com/vanish/2009/11/ff_vanish2...,Writer Evan Ratliff Tried to Vanish: Here's Wh...,"""August 13\, 6:40 PM: I'm driving East out of ...",www.wired.com,2016-12-10 05:00:00+00,2016-12-10 05:00:00+00,1,technology,custom,92,0
...,...,...,...,...,...,...,...,...,...,...,...,...
78815,135370,https://gitlab.com/dsbowen/conditional-inferen...,examples/bayes_primer.ipynb · master · dsbowen...,\N,\N,2021-11-03 06:44:06.529328,2021-11-03 06:44:06.529341,\N,\N,Custom,93,0
78816,135371,https://www.eurogroupforanimals.org/news/membe...,Members States call for ambitious animal welfa...,\N,\N,2021-11-03 06:44:06.539342,2021-11-03 06:44:06.539354,\N,\N,Custom,26,0
78817,135372,https://chrisblattman.com/2007/12/12/how-to-ge...,How to get a PhD *and* save the world - Chris ...,\N,\N,2021-11-03 06:44:06.551245,2021-11-03 06:44:06.551256,\N,\N,Custom,86,0
78818,135373,https://thebulletin.org/2021/09/chinas-nuclear...,China’s nuclear missile silo expansion: From m...,\N,\N,2021-11-03 06:44:06.561783,2021-11-03 06:44:06.561796,\N,\N,Custom,47,0


In [8]:
links.to_csv('data/export.csv', index=False)